# Codes from Chapter 2 - Section 3 - Ascher and Greif - Second Edition
## Example 2.9
Let us revisit Example 2.3, f (x) = x + ln(x) = 0. There are several ways to rewrite the equation f(x) = 0 as x = g(x) and seek a fixed point. Let us look at two possible ways:

**(i)** $g_1(x)=-\ln (x)$

**(ii)** $g_2(x)=e^{-x}$

For the function $g_1(x)$ the fixed point iterations do not converge, as seen in lecture, however for $g_2(x) = e^{−x}$ things are much better. First, we observe that $g_2$ is continuous and monotonically decreasing on $[0, 1]$ with $g_2 (0) = e^0 = 1$ and $g_2 (1) = e^{−1} = 0.3679$. Secondly, we observe that $g_2^\prime (x) = −e^{−x}$ and therefore $|g^\prime(x)| < 1$ for $0 < x < 1$. Therefore we have a unique fixed point in the interval as per the Fixed-Point Theorem.

In [9]:
using Plots

# Perform fixed-point iterations
function fixed_point_iterations(g, x_initial, atol)
    x_vals=[Float64(x_initial)]
    x1=x_initial
    x0=x1-1
    # Store initial value
    while abs(x0-x1) > atol
        x0=x1
        x1 = g(x0)  # Apply the function g
        push!(x_vals,x1)
    end
    return x_vals
end

# Plot y = x, y = g(x), and the fixed-point iterations
function plot_fixed_point(g, x0, atol, x_min, x_max)
    # Generate x values for plotting the curves
    xs = range(x_min, x_max, length=100)
    
    # Perform fixed-point iterations
    x_vals = fixed_point_iterations(g, x0, atol)
    max_iters=length(x_vals)-1

    # Create the plot
    plot(xs, xs, label="y = x", xlabel="x", ylabel="y", lw=2)  # Plot y = x
    plot!(xs, g.(xs), label="y = e^{-x}", lw=2)                # Plot y = e^(-x)
    
    # Plot the iterations
    for i in 1:max_iters
        plot!([x_vals[i], x_vals[i]], [x_vals[i], x_vals[i+1]], color=:red, lw=1, label=false)  # Vertical line
        plot!([x_vals[i], x_vals[i+1]], [x_vals[i+1], x_vals[i+1]], color=:blue, lw=1, label=false)  # Horizontal line
    end
    
    # Mark the iterations with points
    scatter!(x_vals, g.(x_vals), label="Iterations", color=:black, marker=:circle)
end



In [ ]:
# Parameters for the fixed-point iteration
x0 = 1.0  # Initial guess
atol = 1e-6  #absolute tolerance
x_min = 0  # Min x value for the plot
x_max = 1.1  # Max x value for the plot

# Define the function g(x) = e^(-x)
g2(x) = exp(-x)

# Call the plot function
plot_fixed_point(g2, x0, atol, x_min, x_max)

## Example 2.10. 

Consider once more the problem
$$f (x) = 2 \cosh\left(\frac{x}{4}\right) − x = 0.$$

We saw in Example 2.4 that there are two roots. Let us attempt to solve the problem using a fixed-point iteration. Setting

$$g(x) = 2 \cosh\left(\frac{x}{4}\right)$$

Let's plot it first to see its fixed points.

In [ ]:
g(x)=2cosh(x/4)

xs = range(0, 10, length=100)
# Create the plot
plot(xs, xs, label="y = x", xlabel="x", ylabel="y", lw=2)  # Plot y = x
plot!(xs, g.(xs), label="y = 2cosh(x/4)", lw=2)                # Plot y = 2cosh(x/4)

there are two fixed points (i.e., roots of $f(x) = 0$) – let’s call them $x_1$ and $x_2$ – but also that we can bracket them

$$ 2\leq x_1^*\leq 4$$

$$ 8\leq x_2^*\leq 10$$

In [ ]:
# Define the recursive bisection function
function bisection_recursive(func, a, b, fa, fb, atol, ftol)
    p = (a + b) / 2  # Midpoint

    # Base case: If the interval is smaller than the tolerance
    if (b - a) < 2 * atol
        return p
    else
        fp = func(p)  # Evaluate the function at the midpoint

        # Base case: If the function value is smaller than the tolerance
        if abs(fp) < ftol
            return p
        end

        # Recursive case: Narrow down the interval
        if fa * fp < 0
            b = p
            fb = fp
        else
            a = p
            fa = fp
        end

        # Recursive call
        return bisection_recursive(func, a, b, fa, fb, atol, ftol)
    end
end

In [ ]:
f(x)=x-2cosh(x/4)
x1_exact=bisection_recursive(f,2,4,f(2),f(4),1e-8,1e-8)

### Exercise in class

(a) Call **bisection** method from section2_2.ipynb on $f(x)=\cosh\left(\frac{x}{4}\right) − x $ for calculating $x_2^*$ correct to an absolute tolerance 1.e-8 with $[8,10]$.

(b)  Use the same function $g$, for defining the iteration $x_{k+1} =2\cosh\left(\frac{x_k}{4}\right)$ and call the "fixed_point_iterations" function for $g$ with the following initial points and atol=1e-10:
    (i) $x_0=4$
    (ii) $x_0=10$
    (iii) $x_0=8$


## Example 2.11

In [ ]:
function newton(f,fp,x_curr,tol)
    x_old=x_curr-1
    i=0
    x_vals=[Float64(x_curr)]
    while (abs(x_old-x_curr)>tol)
        i=i+1
        x_old=x_curr
        println("iteration:",i)
        x_curr=x_old-(f(x_old)/fp(x_old))
        println("x_curr=$x_curr, f(x_curr)=",f(x_curr))
        push!(x_vals,x_curr)
    end
    return x_curr,x_vals
end
function secant(f,x0,x1,tol)
    f0=f(x0)
    f1=f(x1)
    x_new=0
    i=0
    x_vals=[Float64(x0);Float64(x1)]
    while (abs(x0-x1)>tol)
        i=i+1
        println("iteration:",i)
        x_new=x1-((f1*(x1-x0))/(f1-f0))
        x0=x1
        x1=x_new
        f0=f1
        f1=f(x1)
        println("x_new=",x_new)
        push!(x_vals,x1)
    end
    return x_new,x_vals
end

In [ ]:
fp(x)=1-0.5sinh(x/4)
println("Newton's")
x_newton, x_vals_newton = newton(f,fp,2,1e-10)
println("Secant")
x_secant, x_vals_secant = secant(f,4,2,1e-10)
plot(1:length(x_vals_newton),abs.(x_vals_newton .- x1_exact),label="newton",color=:blue,marker=:circle,yscale=:log10)
plot!(1:length(x_vals_secant),abs.(x_vals_secant .- x1_exact),label="secant",color=:red,marker=:circle,yscale=:log10)



### Exercise for students

Add fixed point iterations for $g(x)$ to this plot with $x_0=2$.